#  Collaborative agent systems where different roles interact to solve complex problems made using Watsonx, crewai and DuckDuckGo search
## Create a concise 1-page market report for {company_name} that summarizes the research, competitive analysis, and highlights the market opportunities.
##### Special thanks to Nishit Dembla Nishit.Dembla@ibm.com for inspiration in his article
##### https://developer.ibm.com/articles/awb-implementing-ai-agents-crewai-langgraph-and-beeai/

In [1]:
from crewai import Agent, Task, Crew, Process, LLM

from crewai.tools import BaseTool
import os
from dotenv import load_dotenv
from langchain.tools import DuckDuckGoSearchRun

# Tool for web search for a given query  using BaseTool implementation
# Need to ensure the class inherits BaseTool and its imported from crewai.tools, one with langchain gives error strangely
class MyCustomDuckDuckGoTool(BaseTool):
    name: str = "DuckDuckGo Search Tool"
    description: str = "Search the web for a given query."

    def _run(self, query: str) -> str:
        duckduckgo_tool = DuckDuckGoSearchRun()
        
        response = duckduckgo_tool.invoke(query)

        return response


# Define LLM Model by using watsonx credentials

In [2]:

# note the environment variable names especially Watsonx api key. crewai uses litellm which gives error if env key names are different. strangely it even changes the env file!
import os
from crewai import LLM

# Load environment variables from .env file
load_dotenv()


# Required
WATSONX_URL=os.getenv("WATSONX_URL")
WATSONX_API_KEY=os.getenv("WATSONX_API_KEY")
WATSONX_APIKEY=os.getenv("WATSONX_API_KEY")
WX_API_KEY = os.getenv("WATSONX_API_KEY")
WATSONX_PROJECT_ID=os.getenv("PROJECT_ID")


# note the model id from crewai providers list
WATSONX_MODEL_ID = "watsonx/meta-llama/llama-3-1-70b-instruct"

os.environ["WATSONX_URL"] = WATSONX_URL  # (required) Base URL of your WatsonX instance
os.environ["WATSONX_APIKEY"] = WATSONX_API_KEY  # IBM cloud API key (required)
os.environ["WATSONX_PROJECT_ID"] = WATSONX_PROJECT_ID  # Project ID of your WatsonX instance (required)

# though the crewai doc says only  model and baseurl are needed but it gives error if project id is not mentioned.
llm = LLM(
    model=WATSONX_MODEL_ID,
    base_url=WATSONX_URL,
    project_id=WATSONX_PROJECT_ID,
    max_tokens=2000,
    temperature=0.7
)

# Define agents with specific roles

In [3]:
from crewai import Agent, Task, Crew, Process

# Define agents with specific roles
researcher = Agent( role="Market Researcher", goal="Find comprehensive information about the target company and its market", 
                   backstory="You're an expert market researcher with 15 years of experience analyzing companies across various industries.",
                    verbose=True, 
                    tools=[MyCustomDuckDuckGoTool()], # tool function needs to be called as a list. strange syntax needed. 
                    allow_delegation=True,
                      llm=llm )  # pass llm even though doc doesn't mention it

analyst = Agent(
    role="Competitive Analyst",
    goal="Analyze the competitive landscape and identify key differentiators",
    backstory="You specialize in competitive analysis with a deep understanding of market positioning and competitive advantages.",
    verbose=True,
   tools=[MyCustomDuckDuckGoTool()],
    allow_delegation=True, 
      llm=llm
)

strategist = Agent(
    role="Market Strategist",
    goal="Identify market opportunities based on research and analysis",
    backstory="You're a veteran strategist who can spot market opportunities others miss.",
    verbose=True,
    allow_delegation=True,
      llm=llm
)

report_writer = Agent(
    role="Report Writer",
    goal="Create concise, actionable market reports",
    backstory="You excel at distilling complex information into clear, compelling reports for business decision-makers.",
    verbose=True,
    allow_delegation=True,
      llm=llm
)


# Define the list of tasks

In [4]:

# Define tasks
research_task = Task(
    description="Research {company_name} thoroughly. Identify their products/services, target market, business model, recent news, and financial performance if available.",
    expected_output="Comprehensive research document on the company",
    agent=researcher
)

competitor_task = Task(
    description="Analyze the top 1-3 competitors of {company_name}. Identify their strengths, weaknesses, and market positioning compared to {company_name}.",
    expected_output="Competitive analysis document",
    agent=analyst,
    context=[research_task]
)

opportunity_task = Task(
    description="Based on the research and competitive analysis, identify 1-3 key market opportunities for {company_name}.",
    expected_output="List of market opportunities with justification",
    agent=strategist,
    context=[research_task, competitor_task]
)

report_task = Task(
    description="Create a concise 1-page market report for {company_name} that summarizes the research, competitive analysis, and highlights the market opportunities.",
    expected_output="1-page market report in markdown format",
    agent=report_writer,
    context=[research_task, competitor_task, opportunity_task]
)



# Create crew listing the agents, tasks and process to be followed for every query

In [5]:
# Create crew
market_research_crew = Crew(
    agents=[researcher, analyst, strategist, report_writer],
    tasks=[research_task, competitor_task, opportunity_task, report_task],
    process=Process.sequential,
    verbose = True
)


# Ask query to the crew and trigger agent collaboration to give correct result
### takes about 5 minutes for the whole process to complete. Keep verbose True to see the interaction

In [6]:

# Execute the crew
result = market_research_crew.kickoff(inputs={"company_name": "Tata Steel"})
print(result)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: c0f55ac0-a537-4606-90af-bc657e39e22b                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: cefb2f1b-9ed2-4e15-add6-7a3738d8abe4
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: cefb2f1b-9ed2-4e15-add6-7a3738d8abe4
       Status: Executing Task...
    └── 🤖 Agent: Market Researcher
            Status: In Progress

# Agent: Market Researcher
## Task: Research Tata Steel thoroughly. Identify their products/services, target market, business model, recent news, and financial performance if available.


🤖 Agent: Market Researcher
    Status: In Progress
└── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: cefb2f1b-9ed2-4e15-add6-7a3738d8abe4
│      Assigned to: Market Researcher
│      Status: ✅ Completed
│   ├── 🤖 Agent: Market Researcher
│   │       Status: In Progress
│   ├── 🤖 Agent: Market Strategist
│   │       Status: ✅ Completed
│   └── 🤖 Agent: Market Researcher
│           Status: ✅ Completed
├── 📋 Task: 876b9f71-3849-4d3d-8054-5f6946785483
│      Assigned to: Competitive Analyst
│      Status: ✅ Completed
│   ├── 🤖 Agent: Competitive Analyst
│   │       Status: In Progress
│   ├── 🤖 Agent: Market Researcher
│   │       Status: ✅ Completed
│   ├── 🤖 Agent: Market Researcher
│   │       Status: ✅ Completed
│   └── 🤖 Agent: Competitive Analyst
│           Status: ✅ Completed
├── 📋 Task: c8b792a4-aac5-4b2d-a4b6-e617277de244
│      Assigned to: Market Strategist
│      Status: ✅ Completed
│   ├── 🤖 Agent: Market Strategist
│   │       Status: In Progress
│   ├── 🤖 Agent: Market Researcher
│   │       Status: ✅ Completed
│   ├── 🤖 Agent: Competitive Analyst
│   │       Status: ✅ Completed
│   └── 🤖 Agent: Market Strategist
│           Status: ✅ Completed
└── 📋 Task: 40ca2757-bd12-4b04-acc4-4ce0bd6853df
       Status: Executing Task...
    ├── 🤖 Agent: Report Writer
    │       Status: In Progress
    ├── 🤖 Agent: Market Researcher
    │       Status: ✅ Completed
    ├── 🤖 Agent: Competitive Analyst
    │       Status: ✅ Completed
    └── 🤖 Agent: Report Writer
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: cefb2f1b-9ed2-4e15-add6-7a3738d8abe4
│      Assigned to: Market Researcher
│      Status: ✅ Completed
│   ├── 🤖 Agent: Market Researcher
│   │       Status: In Progress
│   ├── 🤖 Agent: Market Strategist
│   │       Status: ✅ Completed
│   └── 🤖 Agent: Market Researcher
│           Status: ✅ Completed
├── 📋 Task: 876b9f71-3849-4d3d-8054-5f6946785483
│      Assigned to: Competitive Analyst
│      Status: ✅ Completed
│   ├── 🤖 Agent: Competitive Analyst
│   │       Status: In Progress
│   ├── 🤖 Agent: Market Researcher
│   │       Status: ✅ Completed
│   ├── 🤖 Agent: Market Researcher
│   │       Status: ✅ Completed
│   └── 🤖 Agent: Competitive Analyst
│           Status: ✅ Completed
├── 📋 Task: c8b792a4-aac5-4b2d-a4b6-e617277de244
│      Assigned to: Market Strategist
│      Status: ✅ Completed
│   ├── 🤖 Agent: Market Strategist
│   │       Status: In Progress
│   ├── 🤖 Agent: Market Researcher
│   │       Status: ✅ Completed
│   ├── 🤖 Agent: Competitive Analyst
│   │       Status: ✅ Completed
│   └── 🤖 Agent: Market Strategist
│           Status: ✅ Completed
└── 📋 Task: 40ca2757-bd12-4b04-acc4-4ce0bd6853df
       Assigned to: Report Writer
       Status: ✅ Completed
    ├── 🤖 Agent: Report Writer
    │       Status: In Progress
    ├── 🤖 Agent: Market Researcher
    │       Status: ✅ Completed
    ├── 🤖 Agent: Competitive Analyst
    │       Status: ✅ Completed
    └── 🤖 Agent: Report Writer
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 40ca2757-bd12-4b04-acc4-4ce0bd6853df                                                                     │
│  Agent: Report Writer                                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: c0f55ac0-a537-4606-90af-bc657e39e22b                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Comprehensive Market Report for Tata Steel

## Executive Summary

Tata Steel, a leading global steel producer, operates in a dynamic and highly competitive market. Our research has identified key trends, competitive landscape, and opportunities for growth that will inform the company's strategic decisions. This report provides a comprehensive overview of the current market situation, highlighting areas of strength, weakness, and potential for expansion.

## Market Trends

1. **Global Demand:** The global steel market is expected to grow at a CAGR of 3.5% from 2023 to 2028, driven by increasing demand from the construction, automotive, and consumer goods sectors.
2. **Sustainability:** The steel industry is shifting towards more sustainable and environmentally friendly production methods, driven by government regulations and consumer preferences.
3. **Digitalization:** The adoption of digital technologies, such as Industry 4.0 and artificial intelligence, is transforming the steel pro